In [6]:
import numpy as np
from astropy.io import fits
import sep
import matplotlib.pyplot as plt
from scipy import ndimage
import os
import matplotlib

# Replace with path to your FITS file (example: 'example.fits' or the UDF file you provided)
fits_path = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"

hdul = fits.open(fits_path)
data = hdul[0].data.astype('float32')
hdr = hdul[0].header
hdul.close()


In [7]:
# Estimate background
bkg = sep.Background(data)
bkg_image = bkg.back()
data_sub = data - bkg


In [8]:
# Detection threshold in sigma (adjust as needed)
thresh = 1.5
objects = sep.extract(data_sub, thresh, err=bkg.globalrms)
print(f"Detected {len(objects)} objects")
# objects is a structured array with fields like x, y, a, b, theta, peak, cxx, etc.


Detected 8630 objects


In [9]:
r = 3.0
err = bkg.rms()
fluxes = []
flux_errs = []

for obj in objects:
    x = np.array([obj['x']])
    y = np.array([obj['y']])
    flux, fluxerr, flag = sep.sum_circle(data_sub, x, y, r, err=err)
    fluxes.append(flux[0])
    flux_errs.append(fluxerr[0])


In [10]:
outdir = 'sep_tutorial_outputs'
os.makedirs(outdir, exist_ok=True)

# Figure 1: original image (log stretch)
plt.figure(figsize=(8,8))
plt.title("Original image (log stretch)")
plt.imshow(np.log10(data - np.nanmin(data) + 1), origin='lower')
plt.colorbar(label='log10(counts)')
plt.savefig(os.path.join(outdir, 'figure1_original.png'), dpi=150)
plt.close()

# Figure 2: estimated background
plt.figure(figsize=(8,8))
plt.title("Estimated background")
plt.imshow(bkg_image, origin='lower')
plt.colorbar(label='background')
plt.savefig(os.path.join(outdir, 'figure2_background.png'), dpi=150)
plt.close()

# Figure 3: background-subtracted image with detected-source ellipses
plt.figure(figsize=(8,8))
plt.title("Background-subtracted image with detected sources")
plt.imshow(data_sub, origin='lower', vmax=np.percentile(data_sub, 99), vmin=np.percentile(data_sub, 5))
plt.colorbar(label='counts (background subtracted)')
for obj in objects:
    e = matplotlib.patches.Ellipse((obj['x'], obj['y']), 6*obj['a'], 6*obj['b'],
                                   angle=obj['theta']*180/np.pi, fill=False, linewidth=0.6)
    plt.gca().add_patch(e)
plt.savefig(os.path.join(outdir, 'figure3_detected_sources.png'), dpi=150)
plt.close()

# Figure 4: segmentation map (simple threshold + labeling)
thresh_image = data_sub > (thresh * bkg.globalrms)
labeled, nlabels = ndimage.label(thresh_image)
plt.figure(figsize=(8,8))
plt.title(f"Segmentation map (thr={thresh} sigma) — {nlabels} regions")
plt.imshow(labeled, origin='lower')
plt.colorbar(label='label')
plt.savefig(os.path.join(outdir, 'figure4_segmentation_map.png'), dpi=150)
plt.close()


In [11]:
# Histogram
plt.figure(figsize=(8,5))
plt.hist(fluxes, bins=50)
plt.xlabel('Aperture flux (counts)')
plt.ylabel('Number of sources')
plt.title('Histogram of aperture fluxes (r=3 pix)')
plt.savefig(os.path.join(outdir, 'flux_histogram.png'), dpi=150)
plt.close()

# Basic stats
mean_flux = np.mean(fluxes)
median_flux = np.median(fluxes)
std_flux = np.std(fluxes)
max_idx = np.argmax(fluxes)
max_flux = fluxes[max_idx]
max_x, max_y = objects[max_idx]['x'], objects[max_idx]['y']
sigma_away = (max_flux - mean_flux) / std_flux if std_flux > 0 else np.nan

print("Number of detected sources:", len(objects))
print("Mean flux:", mean_flux)
print("Median flux:", median_flux)
print("Std dev:", std_flux)
print("Max flux:", max_flux, "at (x,y) = ", (max_x, max_y), f"({sigma_away:.2f} sigma above mean)")


Number of detected sources: 8630
Mean flux: 0.36224368456566036
Median flux: 0.03099068409366737
Std dev: 9.248874995245801
Max flux: 807.2972835731507 at (x,y) =  (np.float64(1914.2549094883857), np.float64(1134.3164850742164)) (87.25 sigma above mean)
